In [9]:
import mysql.connector as sql
import pandas as pd
from configparser import ConfigParser


In [30]:
parser = ConfigParser()
parser.read('gcloud.conf')

host  =  parser.get('db', 'host')
db    =  parser.get('db', 'db')
user  =  parser.get('db', 'user')
passw =  parser.get('db', 'user')


db_connection = sql.connect(host=host, database=db, user=user, password=passw)
db_cursor = db_connection.cursor()
query = "SELECT * FROM all_physical limit 100"


In [31]:
pd.read_sql(query, db_connection)

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,EVENT
0,28/12/2015 10:00:00 AM,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.5988,0.000128152,1,1,1,Normal
1,28/12/2015 10:00:01 AM,2.446274,522.886,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.6789,0.000128152,1,1,1,Normal
2,28/12/2015 10:00:02 AM,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6789,0.000128152,1,1,1,Normal
3,28/12/2015 10:00:03 AM,2.53435,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6148,0.000128152,1,1,1,Normal
4,28/12/2015 10:00:04 AM,2.56926,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,...,2,1,250.8812,1.649953,189.5027,0.000128152,1,1,1,Normal
5,28/12/2015 10:00:05 AM,2.609294,523.8673,2,2,1,262.0161,8.394514,328.6337,2.44411,...,2,1,250.753,1.649953,189.5027,0.000128152,1,1,1,Normal
6,28/12/2015 10:00:06 AM,2.637158,524.1028,2,2,1,262.0161,8.394514,328.6337,2.44411,...,2,1,250.5928,1.649953,189.5027,0.000128152,1,1,1,Normal
7,28/12/2015 10:00:07 AM,2.652211,524.2206,2,2,1,262.0161,8.394514,328.6337,2.441803,...,2,1,250.5928,1.649953,189.5027,0.000128152,1,1,1,Normal
8,28/12/2015 10:00:08 AM,2.655735,524.4954,2,2,1,262.0161,8.394514,328.6337,2.441803,...,2,1,250.9132,1.649953,189.5027,0.000128152,1,1,1,Normal
9,28/12/2015 10:00:09 AM,2.64997,524.0636,2,2,1,262.0161,8.394514,328.6337,2.441803,...,2,1,250.9132,1.649953,189.5668,0.000128152,1,1,1,Normal
